## Framework

- Data cleaning and formatting 
- Exploratory data analysis
- Feature engineering and selection
- Compare several machine learning models on a performance metric
- Perform hyper-parameter tuning on the best model 
- Evaluate the best model on the testing set
- Interpret the model results
- Draw conclusions and document work

In [133]:
#load packages

#lm pacakges
from sklearn import tree 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.preprocessing import normalize, scale, Normalizer, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, SVR, LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.dummy import DummyClassifier

#other 
import numpy as np
import pandas as pd
import pickle 
import graphviz
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling
pd.set_option("display.max_colwidth", 200)

import altair as alt
import time

import autotime

In [134]:
#ignore warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

In [135]:
df1 = pd.read_csv('data/01_oscar_data.csv')
df2 = pd.read_csv('data/02_oscar_data.csv')

In [136]:
#df1.profile_report(style={'full_width':True})

- remove all the categorical for wins and nominations so model only is concern with # of nominations and wins 
    - might have to return to this later

In [137]:
df1.head(1)

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
0,2001,Kate & Leopold,tt0035423,PG-13,118,Comedy|Fantasy|Romance,6.4,44.0,An English Duke from 1876 is inadvertedly dragged to modern day New York where he falls for a plucky advertising executive.,66660,47100000.0,2001-12-25,318.0,125.0,2363.0,1,4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,"Best Music, Original Song",1,Best Original Song - Motion Picture,2,Best Original Song - Motion Picture|Best Performance by an Actor in a Motion Picture - Comedy

### Fill best picture nomination 2018

In [138]:
df1.query("movie == 'Roma'")

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
1202,2018,Roma,tt6155172,R,135,Drama,8.1,96.0,A year in the life of a middle-class family's maid in Mexico City in the early 1970s.,54490,NaN,NaN,497.0,290.0,24.0,29,60,No,Yes,No,Yes,No,No,No,Yes,No,No,No,Yes,No,No,No,Yes,10,Best Foreign Language Film of the Year|Best Achievement in Directing|Best Achievement in Cinematography|Best Motion Picture of the Year|Best Performance by an Actress in a Leading Role|Best Perfor...,2,Best Director - Motion Picture|Be

In [139]:
df1.loc[df1['movie'].str.contains('Roma', case=False), 'release_date'] = "2018-11-21"

In [140]:
df1.loc[df1['movie'].str.contains('Roma', case=False), 'gross'] = 5.206600e+07


In [141]:
df1.loc[df1['movie'].str.contains('Roma', case=False), 'release_date.month'] = 11

In [142]:
df1.query("movie == 'Roma'")

,year,movie,movie_id,certificate,duration,genre,rate,metascore,synopsis,votes,gross,release_date,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Oscar_nominated_categories,Golden_Globes_won,Golden_Globes_won_categories,Golden_Globes_nominated,Golden_Globes_nominated_categories,BAFTA_won,BAFTA_won_categories,BAFTA_nominated,BAFTA_nominated_categories,Screen_Actors_Guild_won,Screen_Actors_Guild_won_categories,Screen_Actors_Guild_nominated,Screen_Actors_Guild_nominated_categories,Critics_Choice_won,Critics_Choice_won_categories,Critics_Choice_nominated,Critics_Choice_nominated_categories,Directors_Guild_won,Directors_Guild_won_categories,Directors_Guild_nominated,Directors_Guild_nominated_categories,Producers_Guild_won,Producers_Guild_won_categories,Producers_Guild_nominated,Producers_Guild_nominated_categories,Art_Directors_Guild_won,Art_Directors_Guild_won_categories,Art_Directors_Guild_nominated,Art_Directors_Guild_nominated_categories,Writers_Guild_won,Writers_Guild_won_categories,Writers_Guild_nominated,Writers_Guild_nominated_categories,Costume_Designers_Guild_won,Costume_Designers_Guild_won_categories,Costume_Designers_Guild_nominated,Costume_Designers_Guild_nominated_categories,Online_Film_Television_Association_won,Online_Film_Television_Association_won_categories,Online_Film_Television_Association_nominated,Online_Film_Television_Association_nominated_categories,Online_Film_Critics_Society_won,Online_Film_Critics_Society_won_categories,Online_Film_Critics_Society_nominated,Online_Film_Critics_Society_nominated_categories,People_Choice_won,People_Choice_won_categories,People_Choice_nominated,People_Choice_nominated_categories,London_Critics_Circle_Film_won,London_Critics_Circle_Film_won_categories,London_Critics_Circle_Film_nominated,London_Critics_Circle_Film_nominated_categories,American_Cinema_Editors_won,American_Cinema_Editors_won_categories,American_Cinema_Editors_nominated,American_Cinema_Editors_nominated_categories,Hollywood_Film_won,Hollywood_Film_won_categories,Hollywood_Film_nominated,Hollywood_Film_nominated_categories,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_won_categories,Austin_Film_Critics_Association_nominated,Austin_Film_Critics_Association_nominated_categories,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_won_categories,Denver_Film_Critics_Society_nominated,Denver_Film_Critics_Society_nominated_categories,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_won_categories,Boston_Society_of_Film_Critics_nominated,Boston_Society_of_Film_Critics_nominated_categories,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_won_categories,New_York_Film_Critics_Circle_nominated,New_York_Film_Critics_Circle_nominated_categories,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_won_categories,Los_Angeles_Film_Critics_Association_nominated,Los_Angeles_Film_Critics_Association_nominated_categories,release_date.year,release_date.month,release_date.day-of-month,release_date.day-of-week
1202,2018,Roma,tt6155172,R,135,Drama,8.1,96.0,A year in the life of a middle-class family's maid in Mexico City in the early 1970s.,54490,52066000.0,2018-11-21,497.0,290.0,24.0,29,60,No,Yes,No,Yes,No,No,No,Yes,No,No,No,Yes,No,No,No,Yes,10,Best Foreign Language Film of the Year|Best Achievement in Directing|Best Achievement in Cinematography|Best Motion Picture of the Year|Best Performance by an Actress in a Leading Role|Best Perfor...,2,Best Director - Mot

# Data cleaning and formatting and Exploratory data analysis

In [143]:
df1_temp = df1.drop(['movie_id', 'release_date', 'synopsis', 'release_date.year','release_date.month', 'release_date.day-of-month', 'release_date.day-of-week', 'American_Cinema_Editors_nominated_categories', 'American_Cinema_Editors_won_categories', 'Art_Directors_Guild_nominated_categories', 'Art_Directors_Guild_won_categories', 'Austin_Film_Critics_Association_nominated_categories', 'Austin_Film_Critics_Association_won_categories', 'BAFTA_nominated_categories', 'BAFTA_won_categories', 'Boston_Society_of_Film_Critics_nominated_categories', 'Boston_Society_of_Film_Critics_won_categories', 'Costume_Designers_Guild_nominated_categories', 'Costume_Designers_Guild_won_categories', 'Critics_Choice_nominated_categories','Critics_Choice_won_categories', 'Denver_Film_Critics_Society_nominated_categories', 'Denver_Film_Critics_Society_won_categories', 'Directors_Guild_nominated_categories', 'Directors_Guild_won_categories', 'Golden_Globes_nominated_categories', 'Golden_Globes_won_categories', 'Hollywood_Film_nominated_categories', 'Hollywood_Film_won_categories', 'London_Critics_Circle_Film_nominated_categories', 'London_Critics_Circle_Film_won_categories', 'Los_Angeles_Film_Critics_Association_nominated_categories', 'Los_Angeles_Film_Critics_Association_won_categories', 'New_York_Film_Critics_Circle_nominated_categories', 'New_York_Film_Critics_Circle_won_categories', 'Online_Film_Critics_Society_nominated_categories', 'Online_Film_Critics_Society_won_categories', 'Online_Film_Television_Association_nominated_categories', 'Online_Film_Television_Association_won_categories', 'Oscar_nominated_categories', 'People_Choice_nominated_categories', 'People_Choice_won_categories', 'Producers_Guild_nominated_categories', 'Screen_Actors_Guild_nominated_categories', 'Screen_Actors_Guild_won_categories', 'Writers_Guild_nominated_categories', 'Writers_Guild_won_categories', 'Producers_Guild_won_categories'], axis = 1)

In [144]:
#df1_temp.profile_report(style={'full_width':True})

- Remove Hollywood_Film_won and Hollywood_Film_nominated do to high correlation 
- Can keep Online_Film_Television_Association_nominated and awards_nominations because it does not look like they are related
- give gross and popularity missing values the mediumn value
- remove the missing metascore, certificate, and release_date.month values 
- Deal with the categorical nature of genre using MultiLabelBinarizer

In [145]:
df1_temp = df1_temp.drop(['Hollywood_Film_won', 'Hollywood_Film_nominated'], axis = 1)

In [146]:
df1_temp = df1_temp.dropna(subset=['metascore', 'certificate'])

In [147]:
df1_temp['gross'].fillna((df1_temp['gross'].median()), inplace=True)
df1_temp['popularity'].fillna((df1_temp['popularity'].median()), inplace=True)

In [148]:
df1_temp['genre'] = [x.split('|') for x in df1_temp['genre']]

In [149]:
mlb = MultiLabelBinarizer()
df1_temp = df1_temp.join(pd.DataFrame(mlb.fit_transform(df1_temp.pop('genre')),
                          columns=mlb.classes_,
                          index=df1_temp.index))

In [150]:
# df1_temp.profile_report(style={'full_width':True})

- Fix the histor category history 

In [151]:
df1_temp = df1_temp.reset_index()

In [152]:
df1_temp.query("Histor == 1")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Histor,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
682,694,2014,Selma,PG-13,128,7.5,89.0,64976,52066000.0,214.0,367.0,489.0,2,20,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,2,1,4,0,0,0,0,1,5,0,0,0,0,0,0,0,0,0,1,0,5,0,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [153]:
df1_temp.iloc[682]["History"] = 1

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [154]:
df1_temp = df1_temp.drop(["Histor", "index"], axis=1) 

- fix the certified column to move tv-ma to r and unrated to not rated

In [155]:
df1_temp = df1_temp.reset_index()

In [156]:
df1_temp.query("certificate == 'TV-MA'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
1164,1164,2016,Jim: The James Foley Story,TV-MA,111,7.8,73.0,631,52822418.0,3.0,19.0,1048.0,0,0,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [157]:
df1_temp.loc[df1_temp['certificate'].str.contains('TV-MA', case=False), 'certificate'] = 'R'

In [158]:
df1_temp.iloc[1159]['certificate'] 

'R'

In [159]:
df1_temp.loc[df1_temp['certificate'].str.contains('Unrated', case=False), 'certificate'] = 'Not Rated'

In [160]:
df1_temp.query("certificate == 'Unrated'")

,index,year,movie,certificate,duration,rate,metascore,votes,gross,user_reviews,critic_reviews,popularity,awards_wins,awards_nominations,Oscar_Best_Picture_won,Oscar_Best_Picture_nominated,Oscar_Best_Director_won,Oscar_Best_Director_nominated,Oscar_Best_Actor_won,Oscar_Best_Actor_nominated,Oscar_Best_Actress_won,Oscar_Best_Actress_nominated,Oscar_Best_Supporting_Actor_won,Oscar_Best_Supporting_Actor_nominated,Oscar_Best_Supporting_Actress_won,Oscar_Best_Supporting_Actress_nominated,Oscar_Best_AdaScreen_won,Oscar_Best_AdaScreen_nominated,Oscar_Best_OriScreen_won,Oscar_Best_OriScreen_nominated,Oscar_nominated,Golden_Globes_won,Golden_Globes_nominated,BAFTA_won,BAFTA_nominated,Screen_Actors_Guild_won,Screen_Actors_Guild_nominated,Critics_Choice_won,Critics_Choice_nominated,Directors_Guild_won,Directors_Guild_nominated,Producers_Guild_won,Producers_Guild_nominated,Art_Directors_Guild_won,Art_Directors_Guild_nominated,Writers_Guild_won,Writers_Guild_nominated,Costume_Designers_Guild_won,Costume_Designers_Guild_nominated,Online_Film_Television_Association_won,Online_Film_Television_Association_nominated,Online_Film_Critics_Society_won,Online_Film_Critics_Society_nominated,People_Choice_won,People_Choice_nominated,London_Critics_Circle_Film_won,London_Critics_Circle_Film_nominated,American_Cinema_Editors_won,American_Cinema_Editors_nominated,Austin_Film_Critics_Association_won,Austin_Film_Critics_Association_nominated,Denver_Film_Critics_Society_won,Denver_Film_Critics_Society_nominated,Boston_Society_of_Film_Critics_won,Boston_Society_of_Film_Critics_nominated,New_York_Film_Critics_Circle_won,New_York_Film_Critics_Circle_nominated,Los_Angeles_Film_Critics_Association_won,Los_Angeles_Film_Critics_Association_nominated,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western


In [161]:
df1_temp = df1_temp.dropna()

In [162]:
df_clean = df1_temp

In [163]:
# #structure features 
# df_clean = df_clean.drop(['index'], axis = 1)
# df_clean.head(1)

### Pull the winners of 2018 for back testing 

In [164]:
best_picture_2018_identifiable = df_clean.query("Oscar_Best_Picture_nominated == 'Yes' & year == 2018")
best_director_2018_identifiable = df_clean.query("Oscar_Best_Director_nominated == 'Yes' & year == 2018")
best_actor_2018_identifiable = df_clean.query("Oscar_Best_Actor_nominated == 'Yes' & year == 2018")
best_actress_2018_identifiable = df_clean.query("Oscar_Best_Actress_nominated == 'Yes' & year == 2018")
best_supporting_actor_2018_identifiable = df_clean.query("Oscar_Best_Supporting_Actor_nominated == 'Yes' & year == 2018")
best_supporting_actress_2018_identifiable = df_clean.query("Oscar_Best_Supporting_Actress_nominated == 'Yes' & year == 2018")
best_oriscreen_2018_identifiable = df_clean.query("Oscar_Best_OriScreen_nominated == 'Yes' & year == 2018")
best_adascreen_2018_identifiable = df_clean.query("Oscar_Best_AdaScreen_nominated == 'Yes' & year == 2018")


In [165]:
t = pd.concat([best_actor_2018_identifiable, best_actress_2018_identifiable, best_adascreen_2018_identifiable, best_director_2018_identifiable, best_oriscreen_2018_identifiable, best_picture_2018_identifiable, best_supporting_actor_2018_identifiable, best_supporting_actress_2018_identifiable], axis=0)
best_back_test_2018_identifiable = t.drop_duplicates(keep='last')

df_ml = df_clean.drop(list(best_back_test_2018_identifiable.index))


In [166]:
#final drop of identifable information
df_ml = df_ml.drop(['year', 'movie', 'index'], axis = 1)
best_back_test_2018_unidentifiable = best_back_test_2018_identifiable.drop(['year', 'movie', 'index'], axis = 1)

### Create the X and Ys

In [167]:
X = df_ml.drop(['Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)
y_best_actor = df_ml['Oscar_Best_Actor_won']
y_best_actress = df_ml['Oscar_Best_Actress_won']
y_best_ada_screen = df_ml['Oscar_Best_AdaScreen_won']
y_best_director = df_ml['Oscar_Best_Director_won']
y_best_ori_screen = df_ml['Oscar_Best_OriScreen_won']
y_best_picture = df_ml['Oscar_Best_Picture_won']
y_best_supporting_actor = df_ml['Oscar_Best_Supporting_Actor_won']
y_best_supporting_actress = df_ml['Oscar_Best_Supporting_Actress_won']

### Split X and Y 

In [189]:
#best actor
X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor = train_test_split(X,
                                                                          y_best_actor,
                                                                          test_size=0.2)
#best actoress 
X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress = train_test_split(X,
                                                                          y_best_actress,
                                                                          test_size=0.2)

#best adaptive screen play 
X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen = train_test_split(X,
                                                                          y_best_ada_screen,
                                                                          test_size=0.2)
#best director
X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director = train_test_split(X,
                                                                          y_best_director,
                                                                          test_size=0.2)

#best orginal screen play 
X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen = train_test_split(X,
                                                                          y_best_ori_screen,
                                                                          test_size=0.2)

#best picture 
X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture = train_test_split(X,
                                                                          y_best_picture,
                                                                          test_size=0.2)

#best supporting actor 
X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor = train_test_split(X,
                                                                          y_best_supporting_actor,
                                                                          test_size=0.2)

#best suporting actress 
X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress = train_test_split(X,
                                                                          y_best_supporting_actress,
                                                                          test_size=0.2)


In [190]:
#split 
numeric_features = ['duration', 'rate', 'metascore', 'gross', 'user_reviews', 'critic_reviews', 'popularity']
    
    
categorical_features = ['certificate', 'Oscar_Best_Picture_nominated', 'Oscar_Best_Director_nominated', 'Oscar_Best_Actor_nominated', 'Oscar_Best_Actress_nominated', 'Oscar_Best_Supporting_Actress_nominated', 'Oscar_Best_OriScreen_nominated']

### Processing Pipeline 

In [191]:
# transform the data to standardize the values in the data 
preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), numeric_features),
        ('ohe', OneHotEncoder(drop="first"), categorical_features)])


In [192]:
def get_scores(model, 
                X_train, y_train,
                X_test, y_test, 
                show = True
               ):
    if show: 
        print("Training error:   %.2f" % (1-model.score(X_train, y_train)))
        print("Validation error: %.2f" % (1-model.score(X_test, y_test)))
        print('\n')
    return (1-model.score(X_train, y_train)), (1-model.score(X_test, y_test))

In [193]:
def diff_class_ml(X_train, X_test, y_train, y_test):
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    models = {
          'dummy': DummyClassifier(), 
          'decision tree': DecisionTreeClassifier(),
          'kNN': KNeighborsClassifier(),
          'logistic regression': LogisticRegression(),
          'random forest' : RandomForestClassifier(), 
          'xgboost' : XGBClassifier(),
          'lgbm': LGBMClassifier()
         }

    for model_name, model in models.items():
        t = time.time()
        #print(model_name, ":")    
        clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
        clf.fit(X_train, y_train);
        tr_err, valid_err = get_scores(clf, X_train, y_train, 
                                       X_test, y_test, show = False)
        elapsed_time = time.time() - t
        results_dict[model_name] = [round(tr_err,3), round(valid_err,3), round(elapsed_time,4)]
        #print("Elapsed time: %.1f s" % elapsed_time)
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Train error", "Validation error", "Time in seconds"]
    return results_df

In [194]:
def oscar_predict(X_train, y_train, dataframe, model):
    """
    Shows Classifier and Probability for the Oscar nomination (or other movie)
    
    Parameters
    ----------
    X_train: numpy.ndarray 
        The X training set of the selected catagory 
    y_train: numpy.ndarray
        The y training set the selected catagory 
    dataframe: numpy.ndarray
        The movies that need to be predicted if they won their category 
    model: sklearn classifier model
        The sklearn model such as LogisticRegression() or SVC()
    
    
    """
    # Lets create an empty dictionary to store all the results
    results_dict = {}
    
    movies = list(dataframe.index)
    movies_names = list(dataframe["movie"])
    dataframe_X = dataframe.drop(['year', 'movie', 'index', 'Oscar_Best_Actor_won', 'Oscar_Best_Actress_won', 'Oscar_Best_AdaScreen_won', 'Oscar_Best_Director_won', 'Oscar_Best_OriScreen_won', 'Oscar_Best_Picture_won', "Oscar_Best_Supporting_Actor_won", "Oscar_Best_Supporting_Actress_won"], axis = 1)

    clf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', model)])
    model = clf.fit(X_train, y_train);
        
    for movie, movie_name in zip(movies, movies_names):
        movie_predict = dataframe_X.loc[[movie]]
        classifier = model.predict(movie_predict)
        classifier = classifier[0]
        prob = model.predict_proba(movie_predict)
        prob = prob[0][1]
        results_dict[movie_name] = [classifier, round(prob,5)]
    
    results_df = pd.DataFrame(results_dict).T
    results_df.columns = ["Classifier", "Probability"]
    return results_df

### Base Model Best Picture

In [195]:
diff_class_ml(X_train_best_picture, X_test_best_picture, y_train_best_picture, y_test_best_picture)

,Train error,Validation error,Time in seconds
dummy,0.032,0.034,0.0214
decision tree,0.000,0.021,0.0192
kNN,0.012,0.021,0.0653
logistic regression,0.015,0.013,0.0192
random forest,0.003,0.013,0.0321
xgboost,0.000,0.017,0.1038
lgbm,0.000,0.017,0.0884


In [197]:
#winner was Green Book 
oscar_predict(X_train_best_picture, y_train_best_picture, best_picture_2018_identifiable, LGBMClassifier())

,Classifier,Probability
A Star Is Born,No,0.00024
Bohemian Rhapsody,No,0.00051
Black Panther,No,0.01383
The Favourite,No,0.37287
Roma,Yes,0.66861
Vice,No,0.00243
Green Book,No,0.00011
BlacKkKlansman,No,0.00028


### Base Model Best Actor

In [176]:
diff_class_ml(X_train_best_actor, X_test_best_actor, y_train_best_actor, y_test_best_actor)

,Train error,Validation error,Time in seconds
dummy,0.039,0.026,0.0196
decision tree,0.000,0.013,0.0195
kNN,0.013,0.013,0.0608
logistic regression,0.016,0.013,0.0202
random forest,0.004,0.017,0.0305
xgboost,0.001,0.017,0.1027
lgbm,0.000,0.026,0.0816


### Base Model Best Actress

In [57]:
diff_class_ml(X_train_best_actress, X_test_best_actress, y_train_best_actress, y_test_best_actress)

,Train error,Validation error,Time in seconds
dummy,0.022,0.038,0.0196
decision tree,0.000,0.026,0.0206
kNN,0.010,0.030,0.0627
logistic regression,0.011,0.030,0.0203
RBF SVM,0.011,0.030,0.0206
random forest,0.001,0.030,0.0303
xgboost,0.000,0.034,0.1033
lgbm,0.000,0.026,0.0820


### Base Model Best Adapted Screenplay 

In [58]:
diff_class_ml(X_train_best_ada_screen, X_test_best_ada_screen, y_train_best_ada_screen, y_test_best_ada_screen)

,Train error,Validation error,Time in seconds
dummy,0.021,0.038,0.0205
decision tree,0.000,0.038,0.0238
kNN,0.010,0.026,0.0597
logistic regression,0.012,0.026,0.0191
RBF SVM,0.013,0.026,0.0205
random forest,0.001,0.034,0.0296
xgboost,0.000,0.034,0.1071
lgbm,0.000,0.030,0.0820


### Base Model Best Director

In [59]:
diff_class_ml(X_train_best_director, X_test_best_director, y_train_best_director, y_test_best_director)

,Train error,Validation error,Time in seconds
dummy,0.032,0.043,0.0189
decision tree,0.000,0.038,0.0218
kNN,0.013,0.017,0.0607
logistic regression,0.016,0.017,0.0189
RBF SVM,0.015,0.017,0.0216
random forest,0.004,0.021,0.0314
xgboost,0.000,0.021,0.1034
lgbm,0.000,0.017,0.0876


### Base Model Best Original Screenplay

In [60]:
diff_class_ml(X_train_best_ori_screen, X_test_best_ori_screen, y_train_best_ori_screen, y_test_best_ori_screen)

,Train error,Validation error,Time in seconds
dummy,0.031,0.017,0.0218
decision tree,0.000,0.009,0.0218
kNN,0.012,0.017,0.0670
logistic regression,0.014,0.009,0.0190
RBF SVM,0.017,0.009,0.0211
random forest,0.000,0.009,0.0313
xgboost,0.000,0.013,0.1005
lgbm,0.000,0.013,0.0806


### Base Model Best Supporting Actor

In [61]:
diff_class_ml(X_train_best_supporting_actor, X_test_best_supporting_actor, y_train_best_supporting_actor, y_test_best_supporting_actor)

,Train error,Validation error,Time in seconds
dummy,0.033,0.030,0.0205
decision tree,0.000,0.009,0.0215
kNN,0.018,0.004,0.0618
logistic regression,0.018,0.004,0.0189
RBF SVM,0.018,0.004,0.0227
random forest,0.004,0.004,0.0298
xgboost,0.002,0.004,0.1136
lgbm,0.000,0.004,0.0967


### Base Model Best Supporting Actress

In [62]:
diff_class_ml(X_train_best_supporting_actress, X_test_best_supporting_actress, y_train_best_supporting_actress, y_test_best_supporting_actress)

,Train error,Validation error,Time in seconds
dummy,0.030,0.030,0.0199
decision tree,0.000,0.034,0.0186
kNN,0.014,0.017,0.0708
logistic regression,0.014,0.017,0.0190
RBF SVM,0.015,0.017,0.0227
random forest,0.001,0.017,0.0321
xgboost,0.001,0.017,0.1058
lgbm,0.000,0.026,0.0811
